## 【問題1】機械翻訳の実行とコードリーディング

In [2]:
'''Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm:

- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
        
# Data download:

English to French sentence pairs.
https://www.manythings.org/anki/fra-eng.zip
Lots of neat sentence pairs datasets can be found at:
https://www.manythings.org/anki/

# References:

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''

'Sequence to sequence example in Keras (character-level).\nThis script demonstrates how to implement a basic character-level\nsequence-to-sequence model. We apply it to translating\nshort English sentences into short French sentences,\ncharacter-by-character. Note that it is fairly unusual to\ndo character-level machine translation, as word-level\nmodels are more common in this domain.\n\n# Summary of the algorithm:\n\n- We start with input sequences from a domain (e.g. English sentences)\n    and correspding target sequences from another domain\n    (e.g. French sentences).\n- An encoder LSTM turns input sequences to 2 state vectors\n    (we keep the last LSTM state and discard the outputs).\n- A decoder LSTM is trained to turn the target sequences into\n    the same sequence but offset by one timestep in the future,\n    a training process called "teacher forcing" in this context.\n    Is uses as initial state the state vectors from the encoder.\n    Effectively, the decoder learns t

In [3]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np

In [4]:
batch_size = 64  # Batch size for training.
#epochs = 100  # Number of epochs to train for.
epochs = 1
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
#num_samples = 1000

# Path to the data txt file on disk.
data_path = './fra-eng/fra.txt'

* 訓練時のハイパラの定義
* 対訳コーパスのパスの定義

In [5]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    #input_text, target_text = line.split('\t')
    input_text, target_text, _ = line.split('\t')
    #input_text, target_text = line.split('\t', maxsplit=1)
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

* fra.txt の翻訳元と翻訳先の単語を拾う
* リストに各単語、各文字を入れる

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))

print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)

print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


* 入出力文字のソートを行う
* 入出力文字の、重複を除いた要素数を取得する
* 入出力単語中の、最長の文字数を取得する

In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')


for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
        
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [8]:
print(decoder_input_data.shape)

(10000, 59, 93)


* 入出力文字のインデックス番号を付与した辞書を作成
* (サンプル数、最長文の長さ、トークン数)の型の器を作成し、エンコーダー、デコーダーの入力及びデコーダーの目標文のデータセットを作成する

In [9]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

* keras レイヤーを使いエンコーダーを定義
* エンコーダーのLSTMは状態のみを出力

In [10]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

* keras レイヤーを定義しデコーダーを定義
* デコーダーのLSTMは状態と系列を出力

In [11]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

* モデルを定義

In [12]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=0.2)

# Save model
model.save('s2s.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/1
8000/8000 [==============================] - 127s 16ms/step - loss: 0.8800 - val_loss: 0.8919


/Users/teruitakahiro/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


* 学習

In [ ]:
# Next: inference mode (sampling).

# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

In [ ]:
# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)


decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = decoder_lstm(
                                                                                           decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)


decoder_model = Model(
                                           [decoder_inputs] + decoder_states_inputs,
                                           [decoder_outputs] + decoder_states)

* 予測用モデルの構築
* エンコーダー
  + LSTM は学習済みの LSTM
* デコーダー
  + LSTM は状態の入力を２つ作る
  + Dense は学習済みの Dense

In [37]:
# Reverse-lookup token index to decode sequences back to
# something readable.

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())

reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

* 辞書のkey, value を入れ替える

In [38]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

* 翻訳用の関数を定義する

In [39]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: oos s  e  e s 

-
Input sentence: Go.
Decoded sentence: oos s  e  e s 

-
Input sentence: Go.
Decoded sentence: oos s  e  e s 

-
Input sentence: Hi.
Decoded sentence: To s s  e s  e s 

-
Input sentence: Hi.
Decoded sentence: To s s  e s  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run!
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run.
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run.
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run.
Decoded sentence: Toss s  e  e s 

-
Input sentence: Run.
Decoded sentence: Toss s  e  e s 

-
Input sentence: 

* エンコーダーの入力データを与えて翻訳を行う

## 【問題2】イメージキャプショニングの学習済みモデルの実行

In [47]:
!git clone https://github.com/pdollar/coco.git

Cloning into 'coco'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 12.28 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [6]:
%ls

10_animate.png                     aclImdb_v1.tar.gz
11_animate.png                     coco/
12_animate.png                     fra-eng/
13_animate.png                     pytorch-tutorial/
14_animate.png                     s2s.h5
15_animate.png                     sprint22_RNN_draft.ipynb
1_animate.png                      sprint23_Gated_RNN_draft.ipynb
2_animate.png                      sprint24_seq2seq_draft.ipynb
3_animate.png                      sprint3_linear_reg_draft.ipynb
4_animate.png                      sprint8_ensemble_draft.ipynb
5_animate.png                      sprint9_FNN_draft.ipynb
6_animate.png                      week1_prod_01.ipynb
7_animate.png                      week1_prod_02.ipynb
8_animate.png                      week1_prod_03.ipynb
9_animate.png                      week2_prev_01.ipynb
Sprint6_decision_tree_draft.ipynb  week2_prev_02.ipynb
Week1_01_prev.ipynb                week2_prev_04.ipynb
Week1_03 (1_7) .ipynb              week3_prev_01_draft.ipy

In [14]:
%cd "./pytorch-tutorial/tutorials/03-advanced/image_captioning"

/Users/teruitakahiro/dive/draft/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [15]:
%ls

README.md         data_loader.py    png/              train.py
__pycache__/      download.sh*      requirements.txt
build_vocab.py    model.py          resize.py
data/             models/           sample.py


In [17]:
!python sample.py --image='png/example.png'

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.
<start> a group of giraffes standing next to each other . <end>


## 【問題3】Kerasで動かしたい場合はどうするかを調査

1. PyTorchで、モデルを定義して学習する
2. 学習済みモデルをONNX形式に変換して保存する
3. ONNX形式のモデルをKerasのモデルに変換する